# Reading Video
> Cinelytics uses OpenCV to read video. The main function is `read_video`, where you can flexibly grab either all or different sequences of the frames of any given video.

In [ ]:
#default_exp io

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%matplotlib notebook

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
#export
from videoutils.utils import *

In [ ]:
#export
__all__ = ['read_video', 'read_frames', 'get_target_frames', 'capture', 'as_tensor', 'as_normalised_tensor', 'lapply']

## Reading Video using OpenCV

In [ ]:
#export
def read_video(fname         : Union[str, cv2.VideoCapture],
               start_idx     : Optional[int]=None,
               end_idx       : Optional[int]=None,
               frame_stride  : Optional[int]=None,
               target_frames : Union[tuple, list, int, np.array]=None,
               apply         : Optional[Callable]=None) -> Union[torch.Tensor, list]:
    """
    Flexible video reader where you can grab frames in different ways
    and return as different dtypes.
    
    **Args**
    
    * `fname`        : a `cv2.VideoCapture` object, or path to the video file
    * `start_idx`    : index of the frame where you'd like to start reading the file
    * `end_idx`      : index of the frame where you'd like to end reading the file
    * `frame_stride` : after reading the frame, every `frame_stride`th index will be read
    * `target_frames`: if this arg is used, `start_idx`, `end_idx`, and `frame_stride` are ignored. Can be an 
 
            - `int`            : returns frame at this index
            - `list`/`np.array`: returns frame at index of each element
            - `tuple`          : a tuple like `(start_idx, end_idx, frame_stride)` where 
                                 `frame_stride`is optional. If this is what you want, using the 
                                 other args instead of `target_frames` is more flexible.
    * `apply`         : a function that transforms a single `np.array` of shape `(height, width, channels)`
    """
    cap    = capture(fname)
    if target_frames is None: target_frames = get_target_frames(fname, start_idx, end_idx, frame_stride)
    frames = read_frames(cap, target_frames)
    cap.release()

    if apply is not None:
        if apply == as_tensor or as_normalised_tensor: frames = torch.stack(lapply(frames, apply))
        else: frames = lapply(frames, apply)
    
    return frames

---

## Helper Functions

In [ ]:
#export
def read_frames(cap: cv2.VideoCapture, 
                target_frames: Union[tuple, list, int, np.array]) -> list:
    "Read specific frames from a `cv2.VideoCapture` object"
    if   isinstance(target_frames, tuple) : frame_idxs = np.arange(*target_frames)
    elif isinstance(target_frames, list)  : frame_idxs = target_frames
    elif isinstance(target_frames, int)   : frame_idxs = [target_frames]
    elif isinstance(target_frames, np.ndarray)   : frame_idxs = target_frames 

    frames=[]
    for i in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret==True: frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) # return np.array
        else: break
    return frames

Helper function for `read_video`

---

In [ ]:
#export
def capture(x: Union[str, cv2.VideoCapture]) -> cv2.VideoCapture:
    "Ensure `cv2.VideoCapture` works properly"
    assert isinstance(x, (str, cv2.VideoCapture)), \
    f"Expected `str` or `cv2.VideoCapture` but received {type(x)} "
    cap = cv2.VideoCapture(x) if isinstance(x, str) else x
    assert(cap.isOpened()), f'Failed to open video "{x}"'
    return cap

Helper function for `read_video`

---

In [ ]:
#export
def get_target_frames(file, start, end, stride):
    "flexibly get the indixes of the frames you want to grab"
    num_frames = capture(file).get(cv2.CAP_PROP_FRAME_COUNT)
    if start is not None:
        if end is None: return np.arange(0, num_frames, stride)
        else:           return np.arange(start, end, stride)
    if start is None:
        if end is None: return np.arange(0, num_frames, stride)
        else:           return np.arange(0, end, stride)

Helper function for `read_video`

---

In [ ]:
vid = read_video('files/interstellar-waves-edit.mp4', target_frames=[0,1,2,3,4,5])

len(vid); vid[0].shape

6

(480, 720, 3)

---

In [ ]:
vid  = read_video('files/interstellar-waves-edit.mp4', target_frames=np.arange(100))
vid2 = read_video('files/interstellar-waves-edit.mp4', start_idx=0, end_idx=100, frame_stride=1)
vid3 = read_video('files/interstellar-waves-edit.mp4', target_frames=(0, 100, 1))
vid4 = read_video('files/interstellar-waves-edit.mp4', end_idx=100)

len(vid)
len(vid) == len(vid2) == len(vid3) == len(vid4)

100

True

---

In [ ]:
vid = read_video('files/interstellar-waves-edit.mp4', target_frames=5, apply=as_tensor)

In [ ]:
vid.shape

torch.Size([1, 3, 480, 720])

### Export -

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_utils.ipynb.
Converted 01_io.ipynb.
Converted index.ipynb.
